In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 2.9 MB/s 
     |████████████████████████████████| 3.5 MB 2.9 MB/s 


In [2]:
import numpy as np
import random

In [3]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [4]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=NormalizeFeatures())

Processing...
Done!


In [5]:
data=dataset[0]
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [6]:
from torch_geometric.nn import GATConv
import torch.nn.functional as F

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels=8,p1=0.6,p2=0.6,activation='relu',h1=8,h2=1):
        super().__init__()
        self.p1=p1
        self.p2=p2
        self.conv1 = GATConv(dataset.num_features, hidden_channels,num_heads=h1,dropout=0.6)
        self.conv2 = GATConv(hidden_channels, dataset.num_classes,num_heads=h2)
        self.activation=activation

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.p1, training=self.training)
        x = self.conv1(x, edge_index)
        if self.activation=='relu':
          x = x.relu()
        else:
          x=F.elu(x)
        x = F.dropout(x, p=self.p2, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [7]:
def train(model,optimizer,criterion):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def validate(model,criterion):
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      loss=criterion(out[data.val_mask], data.y[data.val_mask])
      return loss.item()

def test(model,criterion):
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
      return test_acc

In [8]:
import time

In [10]:
test_acc=[]
train_time=[]
for i in [1,12, 123, 1234, 12345, 123456, 1234567, 12345678, 123456789, 1234567890]:
  set_seed(i)
  start=time.process_time()
  model = GAT(hidden_channels=8,p1=0.6,p2=0.6,activation='elu')
  optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-4)
  criterion = torch.nn.CrossEntropyLoss()
  train_loss = train(model,optimizer,criterion)
  best_loss=validate(model,criterion)
  torch.save(model, 'best-model.pt') 
  for epoch in range(1,200):
      train_loss = train(model,optimizer,criterion)
      validate_loss=validate(model,criterion)
      if abs(validate_loss)<abs(best_loss):
          torch.save(model, 'best-model.pt') 
  train_time.append(time.process_time()-start)
  model = torch.load('best-model.pt')
  model.eval()
  test_acc.append(test(model,criterion))
  print(test(model,criterion))

Random seed set as 1
0.677
Random seed set as 12
0.708
Random seed set as 123
0.69
Random seed set as 1234
0.705
Random seed set as 12345
0.685
Random seed set as 123456
0.692
Random seed set as 1234567
0.662
Random seed set as 12345678
0.688
Random seed set as 123456789
0.637
Random seed set as 1234567890
0.669


In [12]:
print("average acuracy: ", sum(test_acc)/len(test_acc))

average acuracy:  0.6813


In [13]:
print("train time: ", train_time)
print("average train time: ",sum(train_time)/len(train_time))

train time:  [22.86896425, 22.782634949000002, 22.900025365999994, 23.188558953000012, 22.609595514999995, 22.76387490399999, 22.714789722999996, 22.717290886, 22.729567419999995, 22.601600856000005]
average train time:  22.787690282199996
